In [7]:
import cv2
import numpy as np
import mediapipe as mp
import math
import time

# ==========================
# Inicialización
# ==========================
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# ==========================
# Lienzo y estados
# ==========================
canvas = None
prev_point = None

# Colores disponibles
DRAW_COLORS = [
    (0, 0, 255),    # Rojo
    (0, 255, 0),    # Verde
    (255, 0, 0),    # Azul
    (0, 255, 255)   # Amarillo
]
color_names = ["ROJO", "VERDE", "AZUL", "AMARILLO"]
color_index = 0
DRAW_COLOR = DRAW_COLORS[color_index]

# Grosor / estilos
THICKNESSES = [3, 6, 12]
thickness_names = ["FINO", "NORMAL", "GRUESO"]
thickness_index = 1
THICKNESS = THICKNESSES[thickness_index]

PINCH_THRESHOLD = 40
SWIPE_THRESHOLD = 40
COOLDOWN = 0.6
last_action_time = 0

prev_index_x = None
prev_index_y = None

# Borrado
ERASER_RADIUS = 25
full_erase_hold_time = 1.5  # segundos
fist_start_time = None

# ==========================
# Funciones auxiliares
# ==========================
def finger_up(lm, tip, pip):
    return lm[tip].y < lm[pip].y

# ==========================
# Loop principal
# ==========================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    if canvas is None:
        canvas = np.zeros_like(frame)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            lm = hand_landmarks.landmark

            ix, iy = int(lm[8].x * w), int(lm[8].y * h)
            tx, ty = int(lm[4].x * w), int(lm[4].y * h)

            distance = math.hypot(ix - tx, iy - ty)

            index_up = finger_up(lm, 8, 6)
            middle_up = finger_up(lm, 12, 10)
            ring_up = finger_up(lm, 16, 14)
            pinky_up = finger_up(lm, 20, 18)

            current_time = time.time()

            # ==========================
            # BORRADO TOTAL → puño cerrado MANTENIDO
            # ==========================
            if not index_up and not middle_up and not ring_up and not pinky_up:
                if fist_start_time is None:
                    fist_start_time = current_time
                elif current_time - fist_start_time > full_erase_hold_time:
                    canvas = np.zeros_like(frame)
                    prev_point = None
                    cv2.putText(frame, "BORRADO TOTAL", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                fist_start_time = None

            # ==========================
            # BORRADO LOCAL → índice + pulgar + medio (pinza abierta)
            # ==========================
            if index_up and middle_up and not ring_up and not pinky_up and distance > PINCH_THRESHOLD:
                cv2.circle(canvas, (ix, iy), ERASER_RADIUS, (0, 0, 0), -1)
                cv2.circle(frame, (ix, iy), ERASER_RADIUS, (255, 255, 255), 2)
                cv2.putText(frame, "BORRADOR", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                prev_point = None

            # ==========================
            # CAMBIO DE COLOR → solo índice + swipe horizontal
            # ==========================
            elif index_up and not middle_up:
                cv2.putText(frame, f"COLOR: {color_names[color_index]}", (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, DRAW_COLOR, 2)

                if prev_index_x is not None:
                    dx = ix - prev_index_x
                    if abs(dx) > SWIPE_THRESHOLD and current_time - last_action_time > COOLDOWN:
                        if dx > 0:
                            color_index = (color_index + 1) % len(DRAW_COLORS)
                        else:
                            color_index = (color_index - 1) % len(DRAW_COLORS)
                        DRAW_COLOR = DRAW_COLORS[color_index]
                        last_action_time = current_time

            # ==========================
            # CAMBIO DE GROSOR → índice + medio + swipe vertical
            # ==========================
            elif index_up and middle_up and not ring_up:
                cv2.putText(frame, f"GROSOR: {thickness_names[thickness_index]}", (30, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                if prev_index_y is not None:
                    dy = iy - prev_index_y
                    if abs(dy) > SWIPE_THRESHOLD and current_time - last_action_time > COOLDOWN:
                        if dy < 0:
                            thickness_index = (thickness_index + 1) % len(THICKNESSES)
                        else:
                            thickness_index = (thickness_index - 1) % len(THICKNESSES)
                        THICKNESS = THICKNESSES[thickness_index]
                        last_action_time = current_time

            # ==========================
            # DIBUJAR → pellizco
            # ==========================
            elif distance < PINCH_THRESHOLD:
                if prev_point is not None:
                    cv2.line(canvas, prev_point, (ix, iy), DRAW_COLOR, THICKNESS)
                prev_point = (ix, iy)
                cv2.circle(frame, (ix, iy), 8, (0, 255, 0), -1)
            else:
                prev_point = None

            prev_index_x = ix
            prev_index_y = iy

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    else:
        prev_point = None
        prev_index_x = None
        prev_index_y = None
        fist_start_time = None

    output = cv2.addWeighted(frame, 1.0, canvas, 1.0, 0)
    cv2.imshow("Virtual Graffiti", output)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()
